# 시행착오

In [6]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [4]:
%ls trimdata

 Volume in drive C has no label.
 Volume Serial Number is 04ED-7805

 Directory of c:\projects\python\apt\trimdata

2021-04-15  오전 01:00    <DIR>          .
2021-04-15  오전 01:00    <DIR>          ..
2021-04-15  오전 12:29         1,600,714 apt200601.xlsx
2021-04-15  오전 12:30         2,634,168 apt200602.xlsx
2021-04-15  오전 12:30         3,389,704 apt200603.xlsx
2021-04-15  오전 12:30         2,942,021 apt200604.xlsx
2021-04-15  오전 12:31         2,780,836 apt200605.xlsx
2021-04-15  오전 12:31         2,219,198 apt200606.xlsx
2021-04-15  오전 12:31         2,036,597 apt200607.xlsx
2021-04-15  오전 12:31         2,531,730 apt200608.xlsx
2021-04-15  오전 12:32         3,993,229 apt200609.xlsx
2021-04-15  오전 12:48         4,785,614 apt200610.xlsx
2021-04-15  오전 12:49         4,761,955 apt200611.xlsx
2021-04-15  오전 12:49         3,362,008 apt200612.xlsx
2021-04-15  오전 12:32         2,388,204 apt200701.xlsx
2021-04-15  오전 12:33         2,159,101 apt200702.xlsx
2021-04-15  오전 12:33         2,711,712 apt200

In [5]:
# df_09 = pd.read_csv("APTdata/아파트(매매)__실거래가_202010.csv",encoding="cp949")
# #에러났음. 왜그렇지? pd import를안함. 
# Parse error. . 이전 강의에서 배웠죠? 근데 sep을 바꿔줬었는데 이번엔 위에 내용이 있어서 안될수도있음.

# 일단 좆된것. csv는 파일지워서 다른이름으로저장하면 손상이되어버림.
# df = pd.read_csv("APTdata/아파트(매매)__실거래가_202010.csv" ,sep='|',encoding="cp949")
df = pd.read_excel("trimdata/apt200601.xlsx" )

In [9]:
df.columns

Index(['Unnamed: 0', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일',
       '거래금액(만원)', '층', '건축년도', '도로명', '해제사유발생일'],
      dtype='object')

In [11]:
# 이전파일 건드리지 않으려면 반드시 copy로
df1 = df.copy()

# datetime으로 만들기

df1["년월"] = df1["계약년월"].copy()
#format=%Y%m 으로 포맷 바꿔주기.
df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

df1["년월"]
# pd.to_datetime?
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   번지          25921 non-null  object        
 3   본번          25921 non-null  int64         
 4   부번          25921 non-null  int64         
 5   단지명         25921 non-null  object        
 6   전용면적(㎡)     25921 non-null  float64       
 7   계약년월        25921 non-null  int64         
 8   계약일         25921 non-null  int64         
 9   거래금액(만원)    25921 non-null  object        
 10  층           25921 non-null  int64         
 11  건축년도        25921 non-null  int64         
 12  도로명         25921 non-null  object        
 13  해제사유발생일     0 non-null      float64       
 14  년월          25921 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(7), object(5)
memory usage: 3.

In [18]:
df1["거래금액(만원)"]
## 왜안돼?
"""
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')
df1["금액"]
"""
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"])

"""
에러가 나는 이유. 아마 콤마때문인거같음.
기본적으로 
 값 중에 몇 개가 실수로 숫자로 된 문자열이 아니라 문자로 된 문자열이 몇 개 포함되어 있다고 해봅시다. 이럴 경우 문자열을 숫자로 파싱할 수 없다면서 ValueError가 발생하는데요,

출처: https://rfriend.tistory.com/470 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]
"""



ValueError: Unable to parse string "         4,200" at position 0

In [22]:
import re

In [30]:
# 문자열에서 제거하기

string = "Hey! What's up, bro?"
new_string = string.replace("!","").replace("?","").replace(",","")
#new_string = ''.join(filter(str.isalnum, string)) 
print(new_string)

Hey What's up bro


In [38]:

"""
# 특정 공백또는 문자 제거하기 
string = "Hey! What's up, bro?"
new_string = string.strip("W")
#df의 컬럼 a에 대해 적용 시킬 때
df['a'].str.strip()
#df 전체에 다 적용시킬 때
df.apply(lambda x: x.str.strip(), axis = 1)
print(new_string)
"""

Hey! What's up, bro?


In [42]:

# ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하가ㅣ 
    # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
df1["금액"] =df1["거래금액(만원)"].copy()
df1["금액"] = df1["금액"].str.replace(",","")
df1["금액"]

0                 4200
1                 7500
2                 8500
3                12500
4                 5998
             ...      
25916             6400
25917             5500
25918             4500
25919             9000
25920             7200
Name: 금액, Length: 25921, dtype: object

In [44]:
df1["금액"] =pd.to_numeric(df1["금액"])
df1["금액"]

0         4200
1         7500
2         8500
3        12500
4         5998
         ...  
25916     6400
25917     5500
25918     4500
25919     9000
25920     7200
Name: 금액, Length: 25921, dtype: int64

In [12]:
#스트링 숫자로 바꿔주기.
#KeyError발생
# df_next["분양가격"] = pd.to_numeric(df_next["분양가격(㎡)"],errors ='coerce')
# 거래금액인데 잘못씀..
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')

df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')

df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   번지          25921 non-null  object        
 3   본번          25921 non-null  int64         
 4   부번          25921 non-null  int64         
 5   단지명         25921 non-null  object        
 6   전용면적(㎡)     25921 non-null  float64       
 7   계약년월        25921 non-null  int64         
 8   계약일         25921 non-null  int64         
 9   거래금액(만원)    25921 non-null  object        
 10  층           25921 non-null  int64         
 11  건축년도        25921 non-null  int64         
 12  도로명         25921 non-null  object        
 13  해제사유발생일     0 non-null      float64       
 14  년월          25921 non-null  datetime64[ns]
 15  금액          6 non-null      float64       
 16  면적          25921 non-

In [14]:
df1["건축년도"]


0        1997
1        1999
2        1999
3        1999
4        1999
         ... 
25916    1994
25917    1994
25918    1994
25919    1994
25920    1994
Name: 건축년도, Length: 25921, dtype: int64

In [ ]:
df.index[0:14]

In [ ]:
#스트링 숫자로 바꿔주기.
#KeyError발생
# df_next["분양가격"] = pd.to_numeric(df_next["분양가격(㎡)"],errors ='coerce')
# 거래금액인데 잘못씀..
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')

df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')

df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1.info()

계획. 
1. 지역명으로 분리 replace사용.

2. 구별로 분리


In [ ]:
df['동찾기'] = df["법정동"].str.contains("창", case=False, na=False).astype(int)
df['동찾기']

In [ ]:
#해제사유등 필요없는것 제거하기.

# axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
# 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
               ,axis=1).copy()


In [ ]:
df2['지역'] = "서울"
df2.info()

In [ ]:
 #지역으로 나누기.
df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
df2["서울"].sum()
df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
df2["대전"].sum()

location =[]
location = ["강원도",     
            "경기도"  ,  
            "경상남도" ,    
            "경상북도"  ,   
            "광주광역"     ,
            "대구광역"  ,
            "대전광역"   ,
            "부산광역"   ,
            "서울특별"    ,
            "세종특별",
             "울산광역"    ,
            "인천광역"    ,
            "전라남도" ,    
            "전라북도"  ,   
            "제주특별"    ,
            "충청남도" ,   
            "충청북도"  ] 

for i in location :
    df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
    df2.loc[(df2[i] ==  1) , '지역'] = i

df2["지역"].value_counts()

In [ ]:
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()



#뭔가잘못됐네.. 뭔가 누락됨. 해결 제주특별 이었음..

In [47]:
df2["세종특별"].sum()

88

In [49]:
df2["강남구"]

0        0
1        0
2        0
3        0
4        0
        ..
25916    0
25917    0
25918    0
25919    0
25920    0
Name: 강남구, Length: 25921, dtype: int32

In [ ]:
###문제 
###문제 
###문제 
###문제 


df2["면적"]

# value error 
# if df2["면적"] <= 85 & df2["면적"] >= 60 :
#     df2["크기"] = "소형 "
#type error 
# if df2["면적"] <= 85
#  df2["크기"] = "소형 "
# if df2["면적"] <= 60
#     df2["크기"] =  1 

# print df2 if df2["면적"] <= 60
# #뭘 못하고있는가? 범주로 쪼개서 새로운 변수 만들기.
#  [x for x in raw_fe.columns if x not in Y_colname+X_remove]

df2["면적"]>= 60 #이건됨 

#이건 타입에러가남. 
# df2["면적"]>= 60  & df2["면적"] <= 85 

# value error 
# df2["면적"]>= 60.0 and   df2["면적"] <= 85.0
df2["크기"] = df2["면적"] <= 85.0 
# if  df2["면적"]>= 60.0 :
#     if  df2["면적"] <= 85.0 :
#         df2["크기"] = "중형"
#     else :
#         df2["크기"] = "대형"
# df2["확인"] = 1 if df2["크기"]==True #안됨



 
#boolean 인덱싱 불리언 인덱싱

# names = np.array[df2["면적"]]
df2["소형"] = df2["면적"]<= 60
df2["대형"] =  df2["면적"]> 85
# if df2["대형"]==False and df2["소형"] ==False :
#     df2["중형"]
    
#df2["면적"]> 60  and  df2["면적"] <85
type(df2["소형"])

df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
df2.loc[(df2['면적'] > 85), '크기'] = '대형'

# https://datatofish.com/if-condition-in-pandas-dataframe/


# df2.count() if df2['크기'] =="대형" 
#질문 '' 와 " " 의 차이? 
#갯수 카운트하기
df2['크기'].value_counts()
df2['제주특별'].value_counts() 
# df2.info()

In [81]:
#강남 송파 서초 3구 따로 묶어보기 ..abs
df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

#Nan 0으로 만들기
df2["강남3구"] = df2["강남3구"].fillna(0).copy()

#확인하기 
df2["강남3구"].value_counts()

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
25916    0.0
25917    0.0
25918    0.0
25919    0.0
25920    0.0
Name: 강남3구, Length: 25921, dtype: float64

In [84]:
df2["강남3구"].value_counts()


0.0    24158
1.0     1763
Name: 강남3구, dtype: int64


# Code Summary



In [4]:
import requests, bs4
import pandas as pd
import tqdm
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [8]:
#2006년 1~10월 
for z in range(1,10):
    # 파일 불러오기 
    df = pd.read_excel("trimdata/apt20060" +str(z)+".xlsx" )

    # 이전파일 건드리지 않으려면 반드시 copy로
    df1 = df.copy()

    # datetime으로 만들기

    df1["년월"] = df1["계약년월"].copy()
    #format=%Y%m 으로 포맷 바꿔주기.
    df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

    df1["년월"]
    # pd.to_datetime?
    # df1.info()


    # ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하기 
        # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
    df1["금액"] =df1["거래금액(만원)"].copy()
    df1["금액"] = df1["금액"].str.replace(",","")


    # 문자열인 값 숫자로 변환하기 
    df1["금액"] =pd.to_numeric(df1["금액"])
    df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
    df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


    #해제사유등 필요없는것 제거하기.

    # axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
    # 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
    df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
                ,axis=1).copy()
    df2


        
    #크기로 나누기
    df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
    df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
    df2.loc[(df2['면적'] > 85), '크기'] = '대형'
    #확인 
    df2['크기'].value_counts()


    #지역으로 나누기.
    df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
    df2["서울"].sum()
    df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
    df2["대전"].sum()

    location =[]
    location = ["강원도",     
                "경기도"  ,  
                "경상남도" ,    
                "경상북도"  ,   
                "광주광역"     ,
                "대구광역"  ,
                "대전광역"   ,
                "부산광역"   ,
                "서울특별"    ,
                "세종특별",
                "울산광역"    ,
                "인천광역"    ,
                "전라남도" ,    
                "전라북도"  ,   
                "제주특별"    ,
                "충청남도" ,   
                "충청북도"  ] 

    for i in location :
        df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
        df2.loc[(df2[i] ==  1) , '지역'] = i


    #확인
    df2["지역"].value_counts()
    df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


    #강남 송파 서초 3구 따로 묶어보기 ..abs
    df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
    df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
    df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
    df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

    #Nan 0으로 만들기
    df2["강남3구"] = df2["강남3구"].fillna(0).copy()

    #확인하기 
    df2["강남3구"].value_counts()


    # 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
    df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

    #결과 확인
    df2.info()

    #수정된 파일 새경로에 저장하기
    df2.to_excel(("monthlydata/apt20060"+str(z)+".xlsx" ))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   건축년도        25921 non-null  int64         
 3   년월          25921 non-null  datetime64[ns]
 4   금액          25921 non-null  int64         
 5   면적          25921 non-null  float64       
 6   크기          25921 non-null  object        
 7   서울          25921 non-null  int32         
 8   대전          25921 non-null  int32         
 9   강원도         25921 non-null  int32         
 10  지역          25921 non-null  object        
 11  경기도         25921 non-null  int32         
 12  경상남도        25921 non-null  int32         
 13  경상북도        25921 non-null  int32         
 14  광주광역        25921 non-null  int32         
 15  대구광역        25921 non-null  int32         
 16  대전광역        25921 non-

KeyboardInterrupt: 

In [91]:
# 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


25921

In [86]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   건축년도        25921 non-null  int64         
 3   년월          25921 non-null  datetime64[ns]
 4   금액          25921 non-null  int64         
 5   면적          25921 non-null  float64       
 6   서울          25921 non-null  int32         
 7   대전          25921 non-null  int32         
 8   강원도         25921 non-null  int32         
 9   지역          25921 non-null  object        
 10  경기도         25921 non-null  int32         
 11  경상남도        25921 non-null  int32         
 12  경상북도        25921 non-null  int32         
 13  광주광역        25921 non-null  int32         
 14  대구광역        25921 non-null  int32         
 15  대전광역        25921 non-null  int32         
 16  부산광역        25921 non-